In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from bayes_opt import BayesianOptimization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasRegressor
from sklearn.inspection import permutation_importance

data = pd.read_excel('3.xlsx')
x = data.iloc[:, :2].values  
y = data.iloc[:, -1].values  

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.1, random_state=42)

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

train_x_3d = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x_3d = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

def create_model(hidden_layers, neurons, learning_rate):
    model = Sequential()
    model.add(LSTM(int(neurons), input_shape=(train_x_3d.shape[1], train_x_3d.shape[2]), activation='relu', return_sequences=(hidden_layers > 1)))
    for _ in range(1, int(hidden_layers)):
        model.add(LSTM(int(neurons), activation='relu', return_sequences=(_ < int(hidden_layers) - 1)))
    model.add(Dense(1, activation='linear'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
    return model

def optimize(hidden_layers, neurons, learning_rate):
    kflod = KFlod(n_splits=10, shuffle=True, random_state=42)
    model = create_model(hidden_layers, neurons, learning_rate)
    model.fit(train_x_3d, train_y, epochs=50, batch_size=32, verbose=0)
    predictions = model.predict(test_x_3d).flatten()
    r2 = r2_score(test_y, predictions)
    return r2

best_r2 = -np.inf
best_params = None

def track_optimize(hidden_layers, neurons, learning_rate):
    global best_r2, best_params
    r2 = optimize(hidden_layers, neurons, learning_rate)
    if r2 > best_r2:
        best_r2 = r2
        best_params = {
            'hidden_layers': hidden_layers,
            'neurons': neurons,
            'learning_rate': learning_rate
        }
    return r2

optimizer = BayesianOptimization(
    f=track_optimize,  
    pbounds={
        'hidden_layers': (1, 3),
        'neurons': (10, 100),
        'learning_rate': (0.0001, 0.1)
    },
    random_state=42,
)

optimizer.set_gp_params(alpha=1e-4)
optimizer.maximize(n_iter=100000, init_points=10)

if best_params is None:
    raise ValueError("最佳参数未正确更新。请检查优化目标函数。")

best_model = KerasRegressor(model=create_model, hidden_layers=best_params['hidden_layers'], neurons=best_params['neurons'], learning_rate=best_params['learning_rate'], epochs=50, batch_size=32, verbose=0)
best_model.fit(train_x_3d, train_y)
train_pred = best_model.predict(train_x_3d)
test_pred = best_model.predict(test_x_3d)

train_mse = mean_squared_error(train_y, train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(train_y, train_pred)
train_r2 = r2_score(train_y, train_pred)

test_mse = mean_squared_error(test_y, test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(test_y, test_pred)
test_r2 = r2_score(test_y, test_pred)

print(f'Train - MSE: {train_mse}, RMSE: {train_rmse}, MAE: {train_mae}, R^2 score: {train_r2}')
print(f'Test - MSE: {test_mse}, RMSE: {test_rmse}, MAE: {test_mae}, R^2 score: {test_r2}')
print(f'Best parameters found: {best_params}')
print(f'Best R^2 score during optimization: {best_r2}')
